# Reinforcement Learning for Quantum Control

In [ ]:
# Install the dependencies
!pip install tensorflow
!pip install -q tf-agents
!pip install qiskit 
!pip install numpy

In [ ]:
!pip3 uninstall protobuf --yes
!pip3 uninstall python-protobuf --yes
!pip install protobuf

## General Structure

The goal of the project is to find an optimal pulse shape for a quantum gate. In this hackathon project, the aim is to do this without a physical model of the noise, but by learning the optimal pulse with reinforcement learning.

In general in Reinforcement Learning (RL), an agent tries to learn an optimal behaviour in an environment it can (partially) observe the results of its actions on. The agent acts on the environment with some action, which then transitions into a new state and the agent receives 'feedback' in terms of a reward. It is now the goal to optimise the reward.
![rl_illustration](https://upload.wikimedia.org/wikipedia/commons/thumb/1/1b/Reinforcement_learning_diagram.svg/497px-Reinforcement_learning_diagram.svg.png)

### Agent: `agent.py`
The agent in this project 'is the control hardware of a qubit'. As actions, it sends EM-pulses onto an imperfect qubit to achieve a certain quantum gate; it can observe the final state of the qubit and the reward is determined in terms of the fidelity of the final state and the theoretically optimal final state.

This is implemented via [tensorflow agents](https://www.tensorflow.org/agents); a number of technical aspects resulting from that are implemented in `agent.py` (e.g. replay buffer, episodes, action distribution, ..).

The pulse as an action is implemented as a piecewise constant function, i.e. the actual values that are learned is a list of amplitudes for a number of minimal time steps. E.g. this is how the agent samples an action from its action distribution, applies it to the environment (ie. the qubit) and receives the reward (fidelty):
```python
    def evaluate(self, tf_agent, eval_py_env):
        eval_env = tf_py_environment.TFPyEnvironment(eval_py_env)
        num_episodes = 1
        fidelity = []
        actions = []
        for _ in range(num_episodes):
            time_step = eval_env.reset()
            while not time_step.is_last():
                action_step = tf_agent.policy.action(time_step)
                time_step = eval_env.step(action_step.action)
                actions.append(action_step.action)
        fidelity, state, pulse_prog = eval_py_env.get_state(actions)
        return state, fidelity, actions, pulse_prog
```

### Environment: `environment.py`

The environment consists of the qubit that is controlled by the pulses, here it is crucial to simulate a qubit on the pulse level - for this **qiskit `PulseSimulator`** is used.

E.g. a pulse is created from the action..
```python
        with pulse.build(name="pulse_programming_in", backend=armonk_backend
        ) as pulse_prog:

            dc = pulse.DriveChannel(0)
            ac = pulse.acquire_channel(0)

            for action in actions:
                pulse.play([action] * self.interval_width, dc)
            pulse.delay(self.interval_width * len(self.actions_list) + 10, ac)
            mem_slot = pulse.measure(0)
```
.. to be used in the simulator:
```python
        backend_sim = PulseSimulator(system_model=armonk_model)
        qobj = assemble(pulse_prog, backend=backend_sim, meas_return="avg", shots=512)
        sim_result = backend_sim.run(qobj).result()
        vector = sim_result.get_statevector()
        fid = state_fidelity(np.array([0, 1]), vector)
```

## Training

in the training, an `Agent` (see `agent.py`) is created and the tensorflow agents framework is used to train the action distribution via reinforcement learning

In [ ]:
from pulseRL.environment import QiskitEnv
import numpy as np
from tf_agents.environments import tf_py_environment
from pulseRL.agent import Agent
from qiskit.visualization import plot_bloch_multivector
import matplotlib as plt

# Learning Parameters
num_iterations = 1000

#In an iteration multiple episodes are collected together and a trajectory is built out of it.
#Later these trajectory is used for learning. Trajectory is added to a replay buffer and analysed together.
collect_episodes_per_iteration = 250  #
replay_buffer_capacity = 2000  


learning_rate = 1e-3  
num_eval_episodes = 2  
eval_interval = 50
num_intervals = 10
interval_length = 60


"""Enviroment which make use of Qiskit Pulse Simulator and pulse builder to simulate 
the dynamics of a qubit under the influence of a pulse. The RL agent interact with this 
environment through action defined as pulse lenght. Here a constant pulse of amplitude 1 
is used and applied for a time "pulse width". "pulse width" is the action that the agent 
takes here. The agent observes the state obtained with the action along with the Fidelity 
to the expected final state. Here initial state is fixed to |0> and target state is |1>

The pulse is designed as follows
  The process time is divided into "num_intervals" of length "interval_length".
  For each interval a constant amplitude of range(0,1) is defined by the agent
  delay the mesearement channel for num_intervals*interval_length + 10 time and make mesurement.
TODO: Make the environement more gernect to handle different operators and initial states"""
environment =  QiskitEnv(np.array([1,0]), num_intervals, interval_length)

#convert the python environment to tensorflow compactible format for training.
tf_dumm = tf_py_environment.TFPyEnvironment(environment)
"""Get the reinfoce agent. Reward is the fielily to target state. Observation is the state"""
agent = Agent(num_iterations, collect_episodes_per_iteration, replay_buffer_capacity, learning_rate, num_eval_episodes, eval_interval, num_intervals, interval_length)
agent_reinforce = agent.get_agent(environment, 'reinforce', "without_noise_trained")
train_results = agent.train(tf_dumm, agent_reinforce)

_(for the algorithm of train() see _`agent.py` _, this mainly uses tf_agent)_

In [ ]:
# useful to save the policy for later usage:
from tf_agents.policies import policy_saver
policy_dir = "policy"
tf_policy_saver = policy_saver.PolicySaver(agent_reinforce.policy)
tf_policy_saver.save(policy_dir)

# Evaluation

In [ ]:
env_test = QiskitEnv(np.array([0,1]),5,100)
vector, fid, action, pulse_prog = agent.evaluate(agent_reinforce, env_test)
print("Fidelity : ", fid)
control_pulse = [act.numpy()[0][0] for act in action]
print("Control pulse", control_pulse)
env_test.get_state(control_pulse)
#plot_bloch_multivector(vector)
pulse_prog.draw()

# Load Best Policy

In [ ]:
import tensorflow as tf 
import numpy as np
from tensorflow.saved_model import load
from tf_agents import policies
from pulseRL.environment import QiskitEnv
import warnings
import logging, sys
def evaluate(policy, eval_py_env):
        eval_env = tf_py_environment.TFPyEnvironment(eval_py_env)
        num_episodes = 1
        fidelity = []
        actions = []
        for _ in range(num_episodes):
            time_step = eval_env.reset()
            while not time_step.is_last():
                action_step = policy.action(time_step)
                time_step = eval_env.step(action_step.action)
                actions.append(action_step.action)
        fidelity,state, pulse_prog = eval_py_env.get_state(actions)
        return state, fidelity, actions,pulse_prog

env_test = QiskitEnv(np.array([0,1]),5,100)
policy_dir = "best_policy"
saved_policy = load(policy_dir)
state, fidelity, actions, pulse_prog = evaluate(saved_policy,env_test)    
print("Showing the results of the best policy")
print("Fidelity : ",fidelity)
print("Initial State: ", [1,0])
print("Final State: ", state)
print("\n\n")
pulse_prog.draw()

In [ ]:
plt.pyplot.plot(train_results[2])